In [18]:
import pandas as pd
import numpy as np
import yfinance as yf
from sklearn.pipeline import Pipeline
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

We download various types of data (Open, High, Low, Close, Adj Close, Volume) for 198 assets between two periods (start, end) into a pandas dataframe denoted as df.

We then store the returns of each of these assets in the 'data'.

Scrapper les tickers sur wikipédia

In [19]:
stock_symbols = ["AAPL", "MSFT", "AMZN", "NVDA", "GOOGL", "TSLA", "META", "UNH", "JPM", "JNJ", "XOM", "V", "PG", "AVGO", "LLY", "HD", "MA", "CVX", "MRK", "ABBV", "PEP", "COST", "ADBE", "KO", "WMT", "BAC", "CSCO", "MCD", "TMO", "CRM", "ACN", "PFE", "NFLX", "CMCSA", "ABT", "LIN", "AMD", "ORCL", "DHR", "WFC", "DIS", "TXN", "PM", "CAT", "INTC", "NEE", "UNP", "INTU", "COP", "VZ", "LOW", "NKE", "QCOM", "UPS", "IBM", "BA", "BMY", "HON", "SPGI", "AMAT", "RTX", "AMGN", "GE", "SBUX", "GS", "MS", "DE", "NOW", "PLD", "MDT", "ELV", "ISRG", "BLK", "BKNG", "ADP", "MDLZ", "LMT", "T", "TJX", "AXP", "SCHW", "ADI", "CVS", "MMC", "GILD", "LRCX", "SYK", "VRTX", "C", "AMT", "ETN", "CI", "CB", "ZTS", "REGN", "SLB", "MO", "TMUS", "FI", "EOG", "BDX", "MU", "PGR", "SO", "BSX", "CME", "PANW", "PYPL", "EQIX", "DUK", "ITW", "KLAC", "SNPS", "CSX", "ATVI", "AON", "SHW", "CL", "NOC", "CDNS", "ICE", "APD", "FCX", "FDX", "TGT", "HUM", "WM", "MMM", "MCK", "MPC", "ORLY", "HCA", "NXPI", "USB", "EMR", "PXD", "PH", "PNC", "CMG", "MCO", "APH", "ROP", "MAR", "NSC", "GM", "F", "GD", "PSX", "MCHP", "FTNT", "CARR", "EW", "MSI", "AJG", "TT", "DXCM", "ADM", "TDG", "AZO", "VLO", "CCI", "PSA", "OXY", "SRE", "ECL", "TEL", "AIG", "ON", "PCAR", "MNST", "ANET", "CHTR", "GIS", "ADSK", "STZ", "NUE", "MSCI", "KMB", "CTAS", "COF", "TFC", "JCI", "AFL", "D", "IDXX", "AEP", "WMB", "WELL", "MET", "EXC", "HLT", "IQV", "PAYX", "EL", "CTVA", "O", "TRV", "ROST"]

n_stocks = len(stock_symbols) # number of stocks = 198

start = "2022-01-01" # start date
end = "2022-12-01" # end date

df = pd.DataFrame(yf.download(stock_symbols, start, end)) # data on the 198 assets
data = np.log(df["Close"]/df["Open"]).transpose() # compute the returns of these assets
data = data.dropna()

[*********************100%%**********************]  198 of 198 completed


In [20]:
data

Date,2022-01-03,2022-01-04,2022-01-05,2022-01-06,2022-01-07,2022-01-10,2022-01-11,2022-01-12,2022-01-13,2022-01-14,...,2022-11-16,2022-11-17,2022-11-18,2022-11-21,2022-11-22,2022-11-23,2022-11-25,2022-11-28,2022-11-29,2022-11-30
AAPL,0.023234,-0.016173,-0.026459,-0.004061,-0.004173,0.018227,0.015890,-0.003356,-0.020635,0.010046,...,-0.002283,0.028876,-0.006719,-0.014422,0.013744,0.010781,-0.001349,-0.006359,-0.021860,0.045822
ABBV,0.000074,-0.001257,0.006424,-0.008615,-0.002739,0.009651,0.008211,0.005624,-0.011023,0.018721,...,-0.010285,0.007851,0.008489,0.010944,0.013229,-0.003507,0.000188,-0.009861,-0.000695,0.014939
ABT,-0.002801,-0.027102,0.000074,0.001036,0.005029,0.003481,0.003470,-0.008712,-0.020914,-0.018017,...,-0.006062,-0.001072,0.003375,0.001445,0.008523,0.009953,0.002715,-0.012541,0.000000,0.023037
ACN,-0.018130,-0.011093,-0.016852,-0.017544,-0.022246,0.026019,0.015041,-0.001748,-0.045574,-0.010361,...,-0.004245,0.004643,-0.014244,0.005310,0.013890,0.004935,0.007110,-0.009112,-0.004608,0.048720
ADBE,-0.004032,-0.017712,-0.039207,0.008046,-0.005779,0.050370,0.008224,-0.001352,-0.029207,0.011806,...,-0.019286,0.018191,-0.039818,-0.024701,0.025684,0.017576,0.004227,-0.020638,0.000275,0.053014
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
WM,-0.022661,0.007944,-0.015390,0.009192,-0.005487,-0.003694,-0.002130,-0.003506,-0.015191,0.006083,...,0.003295,0.008332,0.003916,0.004494,-0.000184,0.004645,-0.001822,0.006583,0.006564,0.018292
WMB,0.014061,0.000374,-0.006323,0.006948,0.018012,-0.004640,0.006254,0.005839,-0.009290,0.012021,...,0.002075,-0.008124,0.015691,0.005748,0.019481,-0.002067,-0.001179,0.004490,0.016452,0.002597
WMT,0.004504,-0.014264,0.007462,-0.000696,0.009640,-0.005723,-0.000069,-0.001672,0.014123,0.005808,...,0.004184,0.006712,0.003735,0.003579,-0.002042,0.002233,0.006226,0.005159,-0.000392,-0.001246
XOM,0.036869,0.027681,0.003752,0.004695,0.005240,-0.007276,0.035086,0.000000,-0.006914,0.017121,...,-0.004683,0.015779,0.010584,0.008870,0.017404,0.011420,-0.005901,-0.007711,-0.000633,-0.001795


In [21]:
def multiple_clusterings(n_repeat, data, model):
  '''
  ## Type of data ##

  n_repeat : integer --> number of time we apply the clustering method
  model : sklearn model we use --> e.g. GaussianMixture()
  data : pd.DataFrame --> data we want to fit to the model

  -------------

  ## Output ##

  Outputs a pandas DataFrame object of shape (len(data.index), n_repeat)

  -------------

  ## Genera idea ##

  The idea is here to train the model on the dataset data multiple time (here n_repeat time)
  and create a DataFrame whose columns are the cluster labels of each stock and whose rows are
  the label of a given stock for each clustering method

  '''
  Y = pd.DataFrame(index=data.index)

  pipeline = Pipeline([
    ('scaler', StandardScaler()),   # Étape de standardisation
    ('kmeans', KMeans(n_clusters=5)) # Étape K-Means avec 3 clusters
  ])
  
  for i in range(n_repeat):
    pipeline.fit(data)
    predicted_labels = pipeline.named_steps['kmeans'].labels_
    data_with_clusters = pd.DataFrame(predicted_labels, index=data.index)
    y_i = "Clustering n°%i" % (i+1)
    Y[y_i] = data_with_clusters

  return Y

In [22]:
pipeline = Pipeline([
    ('scaler', StandardScaler()),   # Étape de standardisation
    ('kmeans', KMeans(n_clusters=5)) # Étape K-Means avec 3 clusters
])

In [23]:
pipeline.fit(data)

/usr/local/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


Pipeline(steps=[('scaler', StandardScaler()), ('kmeans', KMeans(n_clusters=5))])

In [24]:
centroids = pipeline.named_steps['kmeans'].cluster_centers_

centroids.shape

(5, 230)

In [25]:
labels = pipeline.named_steps['kmeans'].labels_

In [26]:
pd.value_counts(labels)

/var/folders/q2/wg5gyfhj2r9cd97zfmckktvw0000gn/T/ipykernel_1381/2812286946.py:1: FutureWarning: pandas.value_counts is deprecated and will be removed in a future version. Use pd.Series(obj).value_counts() instead.
  pd.value_counts(labels)


4    55
2    55
0    44
1    33
3    11
Name: count, dtype: int64

In [42]:
model = KMeans(n_clusters=5)
A = multiple_clusterings(n_repeat=100, data=data, model=model)

/usr/local/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/usr/local/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/usr/local/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/usr/local/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` ex

In [43]:
A

,Clustering n°1,Clustering n°2,Clustering n°3,Clustering n°4,Clustering n°5,Clustering n°6,Clustering n°7,Clustering n°8,Clustering n°9,Clustering n°10,...,Clustering n°91,Clustering n°92,Clustering n°93,Clustering n°94,Clustering n°95,Clustering n°96,Clustering n°97,Clustering n°98,Clustering n°99,Clustering n°100
AAPL,4,3,0,1,2,1,1,2,1,1,...,2,4,2,1,2,2,0,0,2,2
ABBV,0,2,2,2,1,4,4,1,2,0,...,0,2,3,2,0,3,2,1,0,3
ABT,2,0,0,1,1,1,1,4,2,1,...,0,4,3,2,4,0,2,1,2,2
ACN,2,0,0,1,2,1,1,4,1,1,...,3,4,4,4,4,0,0,3,2,2
ADBE,4,3,4,4,2,2,3,2,3,3,...,2,1,2,1,2,2,3,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
WM,0,2,2,2,1,4,4,1,2,0,...,0,2,3,2,0,3,2,1,0,3
WMB,3,2,3,3,4,4,4,3,0,2,...,4,0,0,2,1,4,1,2,0,4
WMT,0,2,2,2,1,4,4,1,2,0,...,0,2,3,2,0,3,2,1,0,3
XOM,3,1,3,3,4,3,2,3,0,2,...,4,0,0,0,1,4,1,2,3,4


In [54]:
len(A.transpose())

101

In [62]:
A.iloc[0, 1]

3

In [63]:
names = ['Cluster 1', 'Cluster 2', 'Cluster 3', 'Cluster 4', 'Cluster 5']
Y = pd.DataFrame(index=names, columns=A.columns)

In [71]:
Y

,Clustering n°1,Clustering n°2,Clustering n°3,Clustering n°4,Clustering n°5,Clustering n°6,Clustering n°7,Clustering n°8,Clustering n°9,Clustering n°10,...,Clustering n°92,Clustering n°93,Clustering n°94,Clustering n°95,Clustering n°96,Clustering n°97,Clustering n°98,Clustering n°99,Clustering n°100,Stock
Cluster 1,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Cluster 2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Cluster 3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Cluster 4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Cluster 5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [70]:
Y.iloc[0, 0]

3

In [75]:
def cluster_composition(multiple_clustering):

    n_clustering = len(multiple_clustering.transpose()) - 1 ## minus 1 because we don't want to take into account the 
                                                            ## first column that corresponds to the tickers name

    names = ['Cluster 1', 'Cluster 2', 'Cluster 3', 'Cluster 4', 'Cluster 5']

    Y = pd.DataFrame(index=names, columns=multiple_clustering.columns)

    for i in range(n_clustering): ## we range across the different clustering so as to recover the clustering composition at each step
        clustering = multiple_clustering.iloc[:, i]
        distinct_values = clustering.unique()

        for k, value in enumerate(distinct_values): 
            l = []
            for j in range(len(stock_symbols)): 
                if multiple_clustering.iloc[j, i] == value:
                    l.append(stock_symbols[j])

            Y.iloc[k, i] = l
    
    return Y

In [76]:
cluster_composition(A)

,Clustering n°1,Clustering n°2,Clustering n°3,Clustering n°4,Clustering n°5,Clustering n°6,Clustering n°7,Clustering n°8,Clustering n°9,Clustering n°10,...,Clustering n°92,Clustering n°93,Clustering n°94,Clustering n°95,Clustering n°96,Clustering n°97,Clustering n°98,Clustering n°99,Clustering n°100,Stock
Cluster 1,"[AAPL, GOOGL, TSLA, JPM, AVGO, LLY, CVX, MRK, ...","[AAPL, GOOGL, TSLA, JPM, AVGO, LLY, CVX, MRK, ...","[AAPL, AMZN, NVDA, UNH, PG, ABBV, PEP, COST, A...","[AAPL, AMZN, NVDA, UNH, V, PG, ABBV, PEP, COST...","[AAPL, NVDA, GOOGL, JPM, CVX, MRK, DHR, INTC, ...","[AAPL, AMZN, NVDA, UNH, V, PG, ABBV, PEP, COST...","[AAPL, AMZN, NVDA, UNH, PG, MA, ABBV, PEP, COS...","[AAPL, GOOGL, JPM, CVX, MRK, CRM, DHR, INTC, C...","[AAPL, NVDA, V, PEP, COST, WMT, CSCO, MCD, CRM...","[AAPL, AMZN, NVDA, UNH, PG, ABBV, PEP, COST, A...",...,"[AAPL, AMZN, NVDA, UNH, V, PG, ABBV, PEP, COST...","[AAPL, GOOGL, JPM, CVX, MRK, DHR, INTC, COP, R...","[AAPL, GOOGL, JPM, CVX, MRK, DHR, INTC, COP, R...","[AAPL, GOOGL, TSLA, JPM, AVGO, LLY, CVX, MRK, ...","[AAPL, GOOGL, TSLA, JPM, AVGO, LLY, CVX, MRK, ...","[AAPL, NVDA, V, PEP, COST, WMT, CSCO, MCD, CRM...","[AAPL, GOOGL, JPM, CVX, MRK, DHR, INTC, COP, R...","[AAPL, AMZN, NVDA, UNH, V, PG, ABBV, PEP, COST...","[AAPL, AMZN, NVDA, UNH, PG, ABBV, PEP, COST, A...",NaN
Cluster 2,"[MSFT, META, JNJ, XOM, HD, MA, ADBE, TMO, PFE,...","[MSFT, JNJ, HD, ADBE, BAC, TMO, PFE, NFLX, AMD...","[MSFT, META, JNJ, XOM, HD, MA, ADBE, BAC, TMO,...","[MSFT, META, JNJ, XOM, HD, MA, ADBE, BAC, TMO,...","[MSFT, AMZN, META, UNH, JNJ, XOM, PG, HD, MA, ...","[MSFT, META, JNJ, XOM, HD, MA, ADBE, BAC, TMO,...","[MSFT, META, JNJ, XOM, HD, ADBE, BAC, TMO, PFE...","[MSFT, META, JNJ, XOM, HD, MA, ADBE, BAC, TMO,...","[MSFT, AMZN, META, UNH, JNJ, XOM, PG, HD, MA, ...","[MSFT, META, JNJ, XOM, HD, MA, ADBE, BAC, TMO,...",...,"[MSFT, META, JNJ, XOM, HD, MA, ADBE, BAC, TMO,...","[MSFT, AMZN, META, UNH, JNJ, XOM, PG, HD, MA, ...","[MSFT, AMZN, META, UNH, JNJ, XOM, PG, HD, MA, ...","[MSFT, META, JNJ, XOM, HD, MA, ADBE, BAC, TMO,...","[MSFT, META, JNJ, XOM, HD, ADBE, BAC, TMO, PFE...","[MSFT, AMZN, META, UNH, JNJ, XOM, PG, HD, MA, ...","[MSFT, AMZN, META, UNH, JNJ, XOM, PG, HD, MA, ...","[MSFT, META, JNJ, XOM, HD, MA, ADBE, BAC, TMO,...","[MSFT, META, JNJ, XOM, HD, MA, ADBE, BAC, TMO,...",NaN
Cluster 3,"[AMZN, NVDA, UNH, PG, ABBV, PEP, COST, BAC, AC...","[AMZN, NVDA, UNH, PG, MA, ABBV, PEP, COST, ACN...","[GOOGL, TSLA, JPM, AVGO, LLY, CVX, MRK, KO, DH...","[GOOGL, JPM, CVX, MRK, DHR, INTC, COP, RTX, AD...","[TSLA, AVGO, LLY, KO, ETN, FI, CME, DUK, WM, E...","[GOOGL, JPM, CVX, MRK, DHR, INTC, COP, RTX, AD...","[GOOGL, TSLA, JPM, AVGO, LLY, CVX, MRK, KO, DH...","[AMZN, NVDA, UNH, V, PG, ABBV, PEP, COST, WMT,...","[GOOGL, JPM, CVX, MRK, DHR, COP, RTX, ADP, SCH...","[GOOGL, TSLA, JPM, AVGO, LLY, CVX, MRK, KO, DH...",...,"[GOOGL, JPM, CVX, MRK, DHR, COP, RTX, ADP, SCH...","[NVDA, V, PEP, COST, WMT, CSCO, MCD, CRM, ACN,...","[NVDA, V, PEP, COST, WMT, CSCO, MCD, CRM, ACN,...","[AMZN, NVDA, UNH, PG, ABBV, PEP, COST, ACN, AB...","[AMZN, NVDA, UNH, PG, MA, ABBV, PEP, COST, ACN...","[GOOGL, JPM, CVX, MRK, DHR, INTC, COP, RTX, AD...","[NVDA, V, PEP, COST, WMT, CSCO, MCD, CRM, ACN,...","[GOOGL, JPM, CVX, MRK, DHR, INTC, COP, RTX, AD...","[GOOGL, TSLA, JPM, AVGO, LLY, CVX, MRK, KO, DH...",NaN
Cluster 4,"[V, WMT, CSCO, MCD, CRM, CMCSA, LIN, NEE, BMY,...","[META, XOM, V, WMT, CSCO, MCD, CRM, CMCSA, LIN...","[V, WMT, CSCO, MCD, CRM, CMCSA, LIN, NEE, BMY,...","[TSLA, AVGO, LLY, KO, ETN, FI, CME, DUK, WM, E...","[V, PEP, COST, WMT, CSCO, MCD, CRM, ACN, CMCSA...","[TSLA, AVGO, LLY, KO, ETN, FI, CME, DUK, WM, E...","[V, WMT, CSCO, MCD, CRM, CMCSA, LIN, NEE, LOW,...","[TSLA, AVGO, LLY, KO, ETN, FI, CME, DUK, WM, E...","[TSLA, AVGO, LLY, KO, ETN, FI, CME, DUK, WM, E...","[V, WMT, CSCO, MCD, CRM, CMCSA, LIN, NEE, BMY,...",...,"[TSLA, AVGO, LLY, KO, ETN, FI, CME, DUK, WM, E...","[TSLA, AVGO, LLY, KO, ETN, FI, CME, DUK, WM, E...","[TSLA, AVGO, LLY, KO, ETN, FI, CME, 

In [ ]:
def 